In [1]:
#Import various python packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import plotly.express as px
import scipy.stats as stats
import scipy.special as sps  
from scipy.stats import gamma

In [2]:
ocph=pd.read_csv("2021-KNYC-METARs.csv",
                parse_dates =['DATE'],
                index_col=['DATE'])

#set time zone as GMT and then convert to EST 
#
ocph.index=ocph.index.tz_localize('GMT')
ocph.index=ocph.index.tz_convert("EDT")
#parsing steps to take 

#1. split column AA1 by commas into seperate columns 
#nh=pd.concat([ocph['DATE'],ocph['AA1'].str.split(',',expand=True)], axis=1)
nh=ocph['AA1'].str.split(',',expand=True)
nh=nh.rename(columns={0:'period',1:'accumulation',2:'condition',3:'quality'})
# period,accumulation,condition,quality

#2. filter for records that are truly hourly... it's all still a string 
nh['period']=nh['period'].str.strip("0")
#nh['accumulation']=nh['accumulation'].str.lstrip("0")
nh
#convert precip nums from str
nh['accumulation']=nh['accumulation'].astype('Int64')
#nh=nh.astype({'period':'Int64','accumulation':'Int64','condition':'Int64','quality':'Int64'})

#only hourly periods of data
onh=nh[nh['period']=='1']
onh=onh.reset_index()

#3. check for any other quality issues
#dataset only has ['1', '5', '6', 'I', 'P'], so we just want to get rid of 6 

#https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf
# LIQUID-PRECIPITATION quality code
#  The code that denotes a quality status of the reported LIQUID-PRECIPITATION data.
#  DOM: A specific domain comprised of the characters in the ASCII character set.
#  0 = Passed gross limits check
#  1 = Passed all quality control checks
#  2 = Suspect
#  3 = Erroneous
#  4 = Passed gross limits check, data originate from an NCEI data source
#  5 = Passed all quality control checks, data originate from an NCEI data source
#  6 = Suspect, data originate from an NCEI data source
#  7 = Erroneous, data originate from an NCEI data source
#  9 = Passed gross limits check if element is present
#  A = Data value flagged as suspect, but accepted as good value
#  I = Data value not originally in data, but inserted by validator
#  M = Manual change made to value based on information provided by NWS or FAA
#  P = Data value not originally flagged as suspect, but replaced by validator
#  R = Data value replaced with value computed by NCEI software
#  U = Data value replaced with edited value

onh.drop(onh[onh['quality']=='6'].index,inplace=True)

#4. multiply precip by 10, confirm units- millimeters? 
onh['accumulation']=onh['accumulation']/10 #turns it into mm 
onh['accumulation']=onh['accumulation']/10 #mm to cm

onh['Precipitation(inches)']=onh['accumulation']*0.393701
onh=onh.set_index('DATE') #this will make the easy plotting work 

#select only values that end with :51
onh['minute']=onh.index.minute
oonh=onh[onh['minute']==51]

C:\Users\carol\AppData\Local\Temp\ipykernel_17908\2905982434.py:1: DtypeWarning: Columns (42,43,44,49,90) have mixed types. Specify dtype option on import or set low_memory=False.
  ocph=pd.read_csv("2021-KNYC-METARs.csv",


In [8]:
np.nanmax(oonh['Precipitation(inches)'])

3.46850581

In [6]:
oonh=ocph
oonh=oonh.drop(columns=['STATION','REPORT_TYPE','SOURCE','HourlyPrecipitation'])

In [11]:
oonh[oonh.index.year==2021]

,period,accumulation,condition,quality,Precipitation(inches),minute
DATE,,,,,,
2021-01-01 00:51:00-05:00,1,0.0,9,5,0.0,51
2021-01-01 01:51:00-05:00,1,0.0,9,5,0.0,51
2021-01-01 02:51:00-05:00,1,0.0,9,5,0.0,51
2021-01-01 03:51:00-05:00,1,0.0,9,5,0.0,51
2021-01-01 04:51:00-05:00,1,0.0,9,5,0.0,51
...,...,...,...,...,...,...
2021-12-31 14:51:00-05:00,1,0.0,9,5,0.0,51
2021-12-31 15:51:00-05:00,1,0.0,9,5,0.0,51
2021-12-31 16:51:00-05:00,1,0.0,9,5,0.0,51
